# Lesson 3 - Calling an A2A Agent using an A2A Client

In this lesson, you will use the A2A Python Client to interact with the Policy Agent you created and activated in the previous lesson. The client handles the protocol details, allowing you to discover the agent's capabilities via its `AgentCard` and send messages to it.

In [ ]:
import os

import httpx
from IPython.display import Markdown, display
from a2a.client import (
    Client,
    ClientConfig,
    ClientFactory,
    create_text_message_object,
)
from a2a.types import Artifact, Message, Task
from a2a.utils.message import get_message_text
from helpers import setup_env, display_terminal, display_agent_card

## 3.1. Check Server Status

First, ensure that your `PolicyAgent` server is running.
- Open the terminal by running the cell below.
- If the agent is still running from the previous lesson, you don't need to do anything.
- If the agent has stopped, type: `uv run a2a_policy_agent.py`

In [ ]:
display_terminal(1)

## 3.2. Setup Client Configuration

Load environment variables to get the host and port of the running agent. You will construct the connection URL from these variables.


In [ ]:
setup_env()

host = os.environ.get("AGENT_HOST", "localhost")
port = os.environ.get("POLICY_AGENT_PORT")
# port = os.environ.get("RESEARCH_AGENT_PORT")

In [ ]:
prompt = "How much would I pay for mental health therapy?"
# prompt = "How do I get mental health therapy?"

## 3.3. Run the Client Interaction

Now you will execute the client interaction flow:
1.  **Connect**: Establish a connection to the agent using `ClientFactory`.
2.  **Discover**: Retrieve the `AgentCard` using `get_card()`.
3.  **Construct**: Create a message object with the user's prompt.
4.  **Send & Receive**: Send the message and asynchronously process the response stream (which may include Tasks, Artifacts, or Messages).


In [ ]:
async with httpx.AsyncClient(timeout=100.0) as httpx_client:
    # Step 1: Create a client
    client: Client = await ClientFactory.connect(
        f"http://{host}:{port}",
        client_config=ClientConfig(
            httpx_client=httpx_client,
        ),
    )

    # Step 2: Discover the agent by fetching its card
    agent_card = await client.get_card()
    display_agent_card(agent_card)

    # Step 3: Create the message using a convenient helper function
    message = create_text_message_object(content=prompt)

    display(Markdown(f"**Sending prompt:** `{prompt}` to the agent..."))

    # Step 4: Send the message and await the final response.
    responses = client.send_message(message)

    text_content = ""

    # Step 5: Process the responses from the agent
    async for response in responses:
        if isinstance(response, Message):
            # The agent replied directly with a final message
            print(f"Message ID: {response.message_id}")
            text_content = get_message_text(response)
        # elif isinstance(response, Artifact):
        #     artifact_id = response.artifact_id
        #     print(f"Artifact ID: {artifact_id}")
        #     text_content = get_message_text(response)
        # response is a ClientEvent
        elif isinstance(response, tuple):
            task: Task = response[0]
            print(f"Task ID: {task.id}")
            if task.artifacts:
                artifact: Artifact = task.artifacts[0]
                print(f"Artifact ID: {artifact.artifact_id}")
                text_content = get_message_text(artifact)

    display(Markdown("### Final Agent Response\n-----"))
    if text_content:
        display(Markdown(text_content))
    else:
        display(
            Markdown(
                "**No final text content received or task did not complete successfully.**"
            )
        )

## 3.4. Resources

- [A2A Python SDK Documentation](https://a2a-protocol.org/latest/sdk/python/api/)


<div style="background-color:#fff6ff; padding:13px; border-width:3px; border-color:#efe6ef; border-style:solid; border-radius:6px">
<p> ⬇ &nbsp; <b>Download Notebooks:</b> 1) click on the <em>"File"</em> option on the top menu of the notebook and then 2) click on <em>"Download as"</em> and select <em>"Notebook (.ipynb)"</em>.</p>
</div>
